In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
import yaml

from pyspark import SparkContext
from pyspark.sql import SparkSession

while not Path("data") in Path(".").iterdir():
    os.chdir("..")

plt.style.use("seaborn-white")
conf_dict = yaml.safe_load(Path("config/conf.yaml").read_text())

sc = SparkContext().getOrCreate()

spark = SparkSession.builder.config("spark.driver.memory", "8g").getOrCreate()

22/06/04 19:23:39 WARN Utils: Your hostname, domvwt-XPS-13-9305 resolves to a loopback address: 127.0.1.1; using 192.168.0.24 instead (on interface wlp164s0)
22/06/04 19:23:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/06/04 19:23:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
entities_df = pd.read_csv("data/raw/offshore-leaks/nodes-entities.csv")

/tmp/ipykernel_136344/2320353097.py:1: DtypeWarning: Columns (2,3,6,7,10,11,12,13,14,15,16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  entities_df = pd.read_csv("data/raw/offshore-leaks/nodes-entities.csv")


In [3]:
entities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814345 entries, 0 to 814344
Data columns (total 21 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   node_id                   814345 non-null  int64  
 1   name                      814316 non-null  object 
 2   original_name             389522 non-null  object 
 3   former_name               6837 non-null    object 
 4   jurisdiction              806781 non-null  object 
 5   jurisdiction_description  806783 non-null  object 
 6   company_type              138752 non-null  object 
 7   address                   299323 non-null  object 
 8   internal_id               389522 non-null  float64
 9   incorporation_date        788471 non-null  object 
 10  inactivation_date         144773 non-null  object 
 11  struck_off_date           343843 non-null  object 
 12  dorm_date                 20207 non-null   object 
 13  status                    358033 non-null  o

In [4]:
entities_df.head().T

,0,1,2,3,4
node_id,10000001,10000002,10000003,10000004,10000005
name,"TIANSHENG INDUSTRY AND TRADING CO., LTD.","NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.","HOTFOCUS CO., LTD.","SKY-BLUE GIFTS & TOYS CO., LTD.",FORTUNEMAKER INVESTMENTS CORPORATION
original_name,"TIANSHENG INDUSTRY AND TRADING CO., LTD.","NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.","HOTFOCUS CO., LTD.","SKY-BLUE GIFTS & TOYS CO., LTD.",FORTUNEMAKER INVESTMENTS CORPORATION
former_name,NaN,NaN,NaN,NaN,NaN
jurisdiction,SAM,SAM,SAM,SAM,SAM
jurisdiction_description,Samoa,Samoa,Samoa,Samoa,Samoa
company_type,NaN,NaN,NaN,NaN,NaN
address,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,LOYAL PORT LIMITED 8/F; CRAWFORD TOWER 99 JERV...
internal_id,1001256.0,1001263.0,1000896.0,1000914.0,1001266.0
incorporation_date,23-MAR-2006,27-MAR-2006,10-JAN-2006,06-JAN-2006,19-APR-2006


In [5]:
entities_df.countries.value_counts()

Malta                                                   84331
British Virgin Islands                                  54933
Hong Kong                                               38906
Barbados                                                38430
Switzerland                                             37940
                                                        ...  
Cayman Islands;French Guiana;United Kingdom                 1
Singapore;Japan                                             1
Bahamas;Canada;Cayman Islands                               1
Bahamas;United Kingdom                                      1
United Arab Emirates;British Virgin Islands;Djibouti        1
Name: countries, Length: 1102, dtype: int64

In [6]:
entities_df.jurisdiction_description.value_counts()[:10]

Bahamas                   209713
British Virgin Islands    172390
Malta                      83939
Saint Kitts and Nevis      70604
Undetermined               53571
Aruba                      49050
Panama                     48691
Barbados                   40846
Seychelles                 16886
Samoa                      15019
Name: jurisdiction_description, dtype: int64

In [7]:
entities_df[entities_df.jurisdiction.str.contains("GBR").fillna(False)]

,node_id,name,original_name,former_name,jurisdiction,jurisdiction_description,company_type,address,internal_id,incorporation_date,...,struck_off_date,dorm_date,status,service_provider,ibcRUC,country_codes,countries,sourceID,valid_until,note
806259,240550263,Humco LLC,NaN,NaN,GBR,United Kingdom,NaN,NaN,NaN,07-May-2019,...,NaN,NaN,Dissolved,NaN,NaN,VGB;KGZ;RUS,British Virgin Islands;Kyrgyzstan;Russia,Pandora Papers - Alpha Consulting,Data is current through 2017.,NaN
809426,240460564,CCRM - COMPANY CONSULTANCY & RISK MANAGEMENT LTD,NaN,NaN,GBR,United Kingdom,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pandora Papers - SFM Corporate Services,Provider data is current through 2017,NaN
809445,240460573,CF MEDIA LTD,NaN,NaN,GBR,United Kingdom,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pandora Papers - SFM Corporate Services,NaN,NaN
809484,240460043,A DEVELOPMENT & MANAGEMENT LTD,NaN,NaN,GBR,United Kingdom,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pandora Papers - SFM Corporate Services,Provider data is current through 2013,NaN
809497,240460599,CLANNAD GLOBAL LTD,NaN,NaN,GBR,United Kingdom,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pandora Papers - SFM Corporate Services,Provider data is current through 2017,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812164,240462826,WAOBS LTD,NaN,NaN,GBR,United Kingdom,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pandora Papers - SFM Corporate Services,Provider data is current through 2012,NaN
812220,240462869,WILLIAMS WALKER LTD,NaN,NaN,GBR,United Kingdom,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pandora Papers - SFM Corporate Services,Provider data is current through 2015,NaN
812396,240461852,NATURAZO LTD,NaN,NaN,GBR,United Kingdom,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pandora Papers - SFM Corporate Services,Provider data is current through 2016,NaN
812428,240461878,NEW ELIOPOLIS LTD,NaN,NaN,GBR,United Kingdom,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pandora Papers - SFM Corporate Services,Provider data is current through 2011,NaN


In [8]:
entities_df[entities_df.country_codes.str.contains("GBR").fillna(False)].iloc[:10, :]

,node_id,name,original_name,former_name,jurisdiction,jurisdiction_description,company_type,address,internal_id,incorporation_date,...,struck_off_date,dorm_date,status,service_provider,ibcRUC,country_codes,countries,sourceID,valid_until,note
226,10000227,SILVERWING PROPERTIES S.A.,SILVERWING PROPERTIES S.A.,NaN,SAM,Samoa,NaN,A MALIDA CHARTERED ACCS [Mr Arif Malida] 66 Mo...,1001815.0,29-SEP-2006,...,NaN,NaN,Active,Mossack Fonseca,R28194,GBR,United Kingdom,Panama Papers,The Panama Papers data is current through 2015,NaN
286,10000286,WATERSIDE PROPERTIES LIMITED,WATERSIDE PROPERTIES LIMITED,NaN,SAM,Samoa,NaN,MOSSACK FONSECA & CO. (U.K.) LIMITED INVISION ...,1002005.0,05-DEC-2006,...,15-FEB-2010,NaN,Relocated in new jurisdiction,Mossack Fonseca,R29177,GBR,United Kingdom,Panama Papers,The Panama Papers data is current through 2015,NaN
319,10000318,LUCULLUS INVESTMENT CORPORATION,LUCULLUS INVESTMENT CORPORATION,NaN,PMA,Panama,NaN,KARL U. SANNE THE OLD RECTORY MIDDLE CHINNOCK ...,10627.0,25-OCT-1988,...,15-JAN-2015,NaN,Defaulted,Mossack Fonseca,72,GBR,United Kingdom,Panama Papers,The Panama Papers data is current through 2015,NaN
389,10000391,PRISMA HOLDINGS LLC,PRISMA HOLDINGS LLC,NaN,NEV,Nevada,NaN,MR. THOMAS SHORTT; BARON GALMOY 12 HANDFORD PL...,120886.0,26-DEC-2006,...,01-JAN-2015,NaN,Dissolved,Mossack Fonseca,E0958552006-6,GBR,United Kingdom,Panama Papers,The Panama Papers data is current through 2015,NaN
410,10000412,ASHDALE LIMITED,ASHDALE LIMITED,NaN,SAM,Samoa,NaN,SAMMAR & CO (ACCOUNTANT [Mr Shajar Farooqui] 1...,1002323.0,27-DEC-2006,...,NaN,NaN,Active,Mossack Fonseca,R29581,GBR,United Kingdom,Panama Papers,The Panama Papers data is current through 2015,NaN
445,10000448,Hawthorn Holdings Ltd.,Hawthorn Holdings Ltd.,NaN,SAM,Samoa,NaN,A1 COMPANY SERVICES LIMITED 788-790 FINCHLEY R...,1005814.0,10-JUN-2011,...,NaN,NaN,Active,Mossack Fonseca,50023,GBR,United Kingdom,Panama Papers,The Panama Papers data is current through 2015,NaN
446,10000449,Acanthus Investments Ltd,Acanthus Investments Ltd,NaN,SAM,Samoa,NaN,A1 COMPANY SERVICES LIMITED 788-790 FINCHLEY R...,1005826.0,10-JUN-2011,...,NaN,NaN,Active,Mossack Fonseca,50022,GBR,United Kingdom,Panama Papers,The Panama Papers data is current through 2015,NaN
536,10000538,XURAN INC.,XURAN INC.,NaN,PMA,Panama,NaN,BAKER & MCKENZIE 100 NEW BRIDGE STREET LONDON ...,10839.0,07-JUN-1983,...,07-SEP-1984,NaN,Dissolved,Mossack Fonseca,NaN,GBR,United Kingdom,Panama Papers,The Panama Papers data is current through 2015,NaN
578,10000580,GRANARIS LLP,GRANARIS LLP,NaN,UK,United Kingdom,NaN,GRANARIS LLP INVISION HOUSE WILBURY WAY HITCHI...,1100022.0,17-FEB-2003,...,17-FEB-2004,NaN,Changed agent,Mossack Fonseca,OC303904,GBR,United Kingdom,Panama Papers,The Panama Papers data is current through 2015,NaN
579,10000581,BARNES & MILES LTD.,BARNES & MILES LTD.,NaN,UK,United Kingdom,NaN,BARNES & MILES LTD. INVISION HOUSE WILBURY WAY...,1100052.0,17-AUG-2000,...,17-AUG-2001,NaN,Changed agent,Mossack Fonseca,04054647,GBR,United Kingdom,Panama Papers,The Panama Papers data is current through 2015,NaN
